In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import keras
from keras.layers import Dense, Dropout, PReLU
from keras.optimizers import Adam

from music_generator.basic.random import generate_dataset
from music_generator.basic.signalproc import SamplingInfo
from music_generator.musical.timing import Tempo
from music_generator.musical.scales import GenericScale
from music_generator.analysis.play import play_mono_as_stereo, play_array
from music_generator.basic.signalproc import mix_at
from music_generator.analysis import preprocessing

from music_generator.musical import scales
import numpy as np
from multiprocessing import Pool
from functools import partial

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sampling_info = SamplingInfo(44100)

In [ ]:
all_roots = scales.chromatic_scale('C')
roots = [n.get_symbol() for n in all_roots.generate(0, 1)]
roots

In [ ]:
def generate_dataset_for_root(root):
    return generate_dataset(n_measures=16,
                            tempo=Tempo(120),
                            scale=GenericScale(root, [0, 2, 3, 5, 7, 8, 10]),
                            sampling_info=sampling_info)
    
with Pool(8) as pool:
    datasets = pool.map(generate_dataset_for_root, roots)

In [ ]:
# Make one big data set and make sure data is of same size
audio_tracks, mix = preprocessing.combine_datasets(datasets)

In [ ]:
mix.shape

In [ ]:
audio_tracks.shape

In [ ]:
# play_mono_as_stereo(mix_at(mix, -audio_tracks[2], at=0))

In [ ]:
n_samples = 1024 * 10
fragment_length = 1024 * 1
input_track = mix
target_track = audio_tracks[2]

In [ ]:
# input_track.shape
# audio_tracks[2].shape

In [ ]:
play_array(input_track, sampling_info.sample_rate, range_secs=[50, 51])
play_array(target_track, sampling_info.sample_rate, range_secs=[50, 51])

# Create training set

In [ ]:

# def create_training_data_set(n_samples, fragment_length, input_track, target_track):
#     max_index = min(len(mix), len(target_track))
#     max_start_index = max_index - fragment_length
    
#     # Selection range
#     selection_ranges = np.random.randint(0, max_start_index, n_samples)
#     selection_ranges = [{'begin': x, 'end': x + fragment_length} for x in selection_ranges]
    
#     x = np.array([mix[sr['begin']:sr['end']] for sr in selection_ranges])
#     y = np.array([target_track[sr['begin']:sr['end']] for sr in selection_ranges])
    
#     return x, y

In [ ]:
x, y = preprocessing.create_training_data_set(n_samples, 
                                              fragment_length, 
                                              input_track, 
                                              target_track)

In [ ]:
# play_array(np.repeat(x[8], 10))

In [ ]:
def x_fade_profile(batch_dim):
    x = np.arange(batch_dim)
    return 1 - abs(x - (batch_dim / 2)) / (batch_dim / 2)

In [ ]:
def model_predict(model, input_track):
    dim = input_shape[0]
    n_batches = int(len(input_track) / dim) - 1
    pred_batches = input_track[0:n_batches*dim].reshape((-1, dim))
    
    pred_batches_shifted = input_track[dim//2:n_batches*dim + dim//2].reshape((-1, dim))
    
    xfp = x_fade_profile(dim)
    
    x0 = np.array([xfp * batch for batch in model.predict(pred_batches)]).reshape(-1)
    x1 = np.array([xfp * batch for batch in model.predict(pred_batches_shifted)]).reshape(-1)
    
    return mix_at(x0, x1, dim//2)
#     np.tanh()

# play_array(model_predict(model, mix), do_wait_done=False)
# play_array(model_predict(model, mix[0:5*44100]))

# Build network

In [ ]:
input_shape = x[0].shape
output_shape = x[1].shape[0]

In [ ]:
model = keras.models.Sequential()
model.add(Dense(1024, input_shape=input_shape))
model.add(PReLU())
model.add(Dense(512))
model.add(PReLU())
model.add(Dense(output_shape))
model.compile(Adam(), 'mse')
play_array(model_predict(model, mix)[0:5*44100], do_wait_done=False)

In [ ]:
while True:
    model.fit(x, y, epochs=2)
    play_array(model_predict(model, mix)[0:5*44100], do_wait_done=True)  
    play_array(mix[0:5*44100])

In [ ]:
play_array(model_predict(model, mix[0:15*44100]))

In [ ]:
play_array(mix[0:15*44100])

In [ ]:
from scipy.io.wavfile import read
wf = read("/Users/marcelraas/dev/test_data/anotherDay.wav", mmap=False)


In [ ]:
def model_predict_2(model, input_track):
    dim = input_shape[0]
    n_batches = int(len(input_track) / dim) - 1
    pred_batches = input_track[0:n_batches*dim].reshape((-1, dim))
    
    return model.predict(pred_batches).reshape(-1)

play_array(model_predict_2(model, wf[1][:,0]/32000), range_secs=[177, 200])

In [ ]:
score_tracks_t, audio_tracks_t, mix_t = generate_dataset(n_measures=64,
                                                         tempo=Tempo(120),
                                                         scale=GenericScale('E', [0, 1, 4, 5, 7, 8, 10]),
                                                         sampling_info=sampling_info)

In [ ]:
play_array(model_predict(model, mix_t[0:15*44100]))
# play_array(model_predict(model, audio_tracks_t[2][0:15*44100]))